# **Nonlinear Optimization**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1CtEj1XBifRGZFkaZxdI_0vEWYWfHiLAS?usp=sharing)

Nonlinear optimization involves finding the best solution to a mathematical programming problem in which the objective function and constraints are not necessarily linear. Because nonlinear models include literally all kinds of models *except* linear ones, it is not surprising that this category is a very broad one, and nonlinear optimization must incorporate a wide variety of approaches to solving problems.<br >
The world is full of systems that do not behave linearly. For example, allowing a tree to grow twice as long does not necessarily double the resulting timber harvest; and tripling the amount of fertilizer applied to a wheat field does not necessarily triple the yield (and might even kill the crop!). In a distributed computing system networked for interprocessor communication, doubling the speed of the processors does not mean that all distributed computations will be completed in half the time, because interactions among processors now could foil the anticipated speedup in throughput.<br >
This chapter examines optimization from a very general point of view. We will consider both unconstrained and constrained models. **Unconstrained optimization** is often dealt with through the use of differential calculus to determine maximum or minimum points of an objective function. Constrained models may present us with systems of equations to be solved. In either case, the classical underlying theories that describe the characteristics of an optimum do not necessarily provide the practical *methods* that are suitable for efficient numerical computation of the desired solutions. Nevertheless, a thorough grasp of the subject of nonlinear optimization requires an understanding of both the mathematical foundations of optimization as well as the algorithms that have been developed for obtaining solutions. This chapter is intended to provide insights from both of these perspectives. We will first look at an example of a nonlinear programming problem formulation.<br >

<ul />

**Example 5.1**

Suppose we want to determine a production schedule over several time periods, where the demand in each period can be met with either products in inventory at the end of the previous period or production during the current period. Let the $T$ time periods be indexed by $i = 1, 2, …, T,$ and let $D_i$ be the known demand at time period $i$. Equipment capacities and material limitations restrict production to at most $E_i$ units during period $i$. The labor force $L_i$ during period $i$ can be adjusted according to demand, but hiring and firing is costly, so a cost $C_L$ is applied to the square of the net change in labor force size from one period to the next. The productivity (number of units produced) of each worker during any period $i$ is given as $P_i$
. The number of units of inventory at the end of period $i$ is $I_i$
, and the cost of carrying a unit of inventory into the next period is $C_I$
. The production scheduling problem is then to determine feasible labor force and inventory 
levels in order to meet demand at minimum total cost. The decision variables are the $L_i$
and $I_i$ for $i = 1, …, T$. The initial labor force and inventory levels are given as $L_0$ and $I_0$, 
respectively. Therefore, we wish to

$$
\begin{aligned}
& \text{minimize} && \sum_{i=1}^{T} C_L (L_i - L_{i-1})^2 + C_I I_i \\
& \text{subject to} && L_i \cdot P_i \le E_i && \text{equipment capacities for } i = 1, \dots, T \\
& && I_{i-1} + L_i \cdot P_i \ge D_i && \text{demand for } i=1, \dots, T \\
& && I_i = I_{i-1} + L_i \cdot P_i - D_i && \text{inventory for } i=1, \dots, T \\
& && L_i, I_i \ge 0 && \text{for } i=1, \dots, T
\end{aligned}
$$

This nonlinear model has a quadratic objective function, but linear constraints, and it happens to involve discrete decision variables. Other nonlinear models may involve continuous processes that are represented by time-integrated functions or flow problems described by differential equations.

</ul>

In [1]:
import numpy as np
from scipy.optimize import minimize, Bounds, LinearConstraint

def solve_production_schedule(T, CL, CI, E, D, P, L0, I0):
    # The decision variables are L_1, ..., L_T and I_1, ..., I_T.
    # We will represent them as a single flat array: [L_1, ..., L_T, I_1, ..., I_T].
    num_vars = 2 * T

    # --- 1. Define the Objective Function ---
    def objective(x):
        # x is the flattened array of decision variables.
        # Unpack L and I arrays from x.
        L = x[:T]
        I = x[T:]

        # Calculate the labor cost term: sum(CL * (L_i - L_{i-1})^2)
        # We need to handle L_0 as a special case.
        L_prev = np.concatenate(([L0], L[:-1]))
        labor_cost = np.sum(CL * (L - L_prev)**2)

        # Calculate the inventory cost term: sum(CI * I_i)
        inventory_cost = np.sum(CI * I)

        return labor_cost + inventory_cost

    # --- 2. Define the Constraints ---
    # The constraints are Ax <= b or Ax = b. We will define the matrices A and vectors b.

    # Constraint 1: Equipment capacities -> L_i * P_i <= E_i
    # L_i * P_i - E_i <= 0  -> (P_i * L_i) - E_i <= 0 -> P_i * L_i <= E_i
    # The constraint matrix A will have non-zero values only for the L variables.
    A_capacity = np.zeros((T, num_vars))
    b_capacity = np.array(E)
    for i in range(T):
        A_capacity[i, i] = P[i] # P_i is the coefficient for L_i
    capacity_constraint = LinearConstraint(A_capacity, -np.inf, b_capacity)

    # Constraint 2: Demand satisfaction -> I_{i-1} + L_i * P_i >= D_i
    # -I_{i-1} - L_i * P_i <= -D_i
    # We need to handle I_0 as a special case.
    A_demand = np.zeros((T, num_vars))
    b_demand = -np.array(D)
    for i in range(T):
        A_demand[i, i] = -P[i] # Coefficient for L_i
        if i > 0:
            A_demand[i, T + i - 1] = -1 # Coefficient for I_{i-1}
        else: # Special case for i=1, I_0 is a constant, so we move it to the right side of the inequality.
            b_demand[i] -= -I0
    demand_constraint = LinearConstraint(A_demand, -np.inf, b_demand)

    # Constraint 3: Inventory balance -> I_i = I_{i-1} + L_i * P_i - D_i
    # I_i - I_{i-1} - L_i * P_i = -D_i
    A_balance = np.zeros((T, num_vars))
    b_balance = -np.array(D)
    for i in range(T):
        A_balance[i, i] = -P[i] # Coefficient for L_i
        A_balance[i, T + i] = 1 # Coefficient for I_i
        if i > 0:
            A_balance[i, T + i - 1] = -1 # Coefficient for I_{i-1}
        else: # Special case for i=1, I_0 is a constant.
            b_balance[i] -= -I0
    balance_constraint = LinearConstraint(A_balance, b_balance, b_balance)

    # Combine all linear constraints
    constraints = [capacity_constraint, demand_constraint, balance_constraint]

    # --- 3. Define the Bounds ---
    # L_i >= 0 and I_i >= 0
    # Lower bounds for all variables are 0.
    # Upper bounds are infinite (or a large number, but np.inf works).
    bounds = Bounds(np.zeros(num_vars), np.inf * np.ones(num_vars))

    # --- 4. Provide an Initial Guess ---
    # A simple initial guess where all variables are 1.0.
    x0 = np.ones(num_vars)

    # --- 5. Run the Optimization ---
    result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)

    return result

if __name__ == '__main__':
    # --- Example Usage ---
    # Define parameters for a 3-period problem
    T_periods = 3
    cost_labor_change = 50.0
    cost_inventory_unit = 10.0
    equipment_capacity = [200, 250, 220] # E1, E2, E3
    demand = [150, 180, 200]           # D1, D2, D3
    production_rate = [1.0, 1.0, 1.0]      # P1, P2, P3 (units per worker)
    initial_labor = 100.0                # L0
    initial_inventory = 50.0             # I0

    # Solve the problem
    solution = solve_production_schedule(T_periods, cost_labor_change, cost_inventory_unit,
                                         equipment_capacity, demand, production_rate,
                                         initial_labor, initial_inventory)

    # Print the results
    if solution.success:
        print("Optimization successful!")
        L_optimized = solution.x[:T_periods]
        I_optimized = solution.x[T_periods:]
        print("\nOptimal Labor Force (L):")
        print(f"L0: {initial_labor:.2f}")
        for i in range(T_periods):
            print(f"L{i+1}: {L_optimized[i]:.2f}")

        print("\nOptimal Inventory (I):")
        print(f"I0: {initial_inventory:.2f}")
        for i in range(T_periods):
            print(f"I{i+1}: {I_optimized[i]:.2f}")

        print(f"\nTotal Minimum Cost: {solution.fun:.2f}")
    else:
        print("Optimization failed.")
        print(f"Reason: {solution.message}")

Optimization successful!

Optimal Labor Force (L):
L0: 100.00
L1: 138.51
L2: 164.28
L3: 177.21

Optimal Inventory (I):
I0: 50.00
I1: 38.51
I2: 22.79
I3: 0.00

Total Minimum Cost: 116327.89
